https://medium.com/@bavalpreetsinghh/llamaindex-chunking-strategies-for-large-language-models-part-1-ded1218cfd30

In [ ]:
from rag.document_parse import docs_parse
from src.rag.data_to_milvus import load_nodes_to_milvus
from src.rag.data_to_milvus import create_milvus_collection, insert_embeddings_to_milvus
from src.config import config
import os

In [ ]:
# Load Markdown files
md_files = []
md_files_dir = config.get_path('dataset', 'preprocessed_md')
for root, dirs, files in os.walk(md_files_dir):
    for file in files:
        if file.endswith('.md'):
            md_files.append(os.path.join(root, file))

len(md_files)
# md_file = '/Users/swgj/Documents/Code/python_project/FinanceRAG/data/markdown/0b46f7a2d67b5b59ad67cafffa0e12a9f0837790/0b46f7a2d67b5b59ad67cafffa0e12a9f0837790.md'
# text, doc_ids = load_texts(md_fir)

In [ ]:
md_files[15]

In [ ]:
nodes = docs_parse(md_files)

In [ ]:
len(nodes)

In [ ]:
# for node in nodes:
#     if node.id_ == '0934726b-2796-4725-bdcd-6faa389ed2a7':
#         print(dict(node))

In [ ]:
dict(nodes[10])

In [ ]:
index = load_nodes_to_milvus(nodes)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
Settings.llm = Ollama(model="qwen2.5")
query_engine = index.as_chat_engine()

In [ ]:
res = query_engine.query("中国铁路通信信号有限公司发行前每股净资产为多少？")

In [ ]:
print(res)

In [ ]:
print(dict(res.source_nodes[0]))

In [ ]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.vector_stores.milvus import MilvusVectorStore


In [ ]:
index.storage_context.persist(persist_dir='data/milvus')

In [ ]:
DeepLakeVectorStore('data/milvus_deep')

In [ ]:
# pkl save
import pickle
with open('data/milvus.pkl', 'wb') as f:
    pickle.dump(index, f)

In [ ]:
from pymilvus import MilvusClient
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import StorageContext

# initialize client
client = MilvusClient(uri='data/milvus_md.db')
# db = chromadb.PersistentClient(path="./chroma_db")



In [ ]:
# get collection
# chroma_collection = db.get_or_create_collection("quickstart")
# collection = client.get(collection_name='llamacollection')

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from src.rag.data_to_milvus import embedding_model
from src.config import config

Settings.llm = Ollama(model="qwen2.5")
Settings.embed_model = embedding_model(config.get_path('models','embedding'))
# query_engine = index.as_chat_engine()

# assign chroma as the vector_store to the context
vector_store = MilvusVectorStore(uri='data/milvus_md.db')
# vector_store = MilvusVectorStore(uri='data/milvus_md.db', collection_name='llamacollection')
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load your index from stored vectors
index_new = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [ ]:
# import pickle
# with open('data/milvus_new.pkl', 'wb') as f:
#     pickle.dump(index_new, f)


In [ ]:
# create a query engine
query_engine = index_new.as_query_engine()
# response = query_engine.query("介绍一下武汉力源信息技术股份有限公司?")
response = query_engine.query("中国铁路通信信号有限公司发行前每股净资产为多少？")
print(response)

In [ ]:
response.get_formatted_sources()

In [ ]:
res = query_engine.query("中国铁路通信信号有限公司的介绍")
print(res)

In [ ]:
len(res.source_nodes)

# model test

In [ ]:
from src.sql.sql import get_query_engine

query_engine = get_query_engine()

In [ ]:
query = "港股票日行情中哪一个股票的最高价最高？"
res = query_engine.query(query)
print(res)

In [ ]:
query = "港股票日行情中哪一个股票的最高价最高？"
res = query_engine.query(query)
print(res)

In [ ]:
# for key, value in (query_engine.get_prompts()).items():
#     print("="*10)
#     print(key+'\n')
#     print(value)

In [ ]:
for key, value in (query_engine.get_prompts()).items():
    print('\n')
    print(key)
    for k, v in dict(value).items():
        print(k, v)

In [ ]:
# import json
# from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
# for key, value in dict(DEFAULT_TEXT_TO_SQL_PROMPT).items():
#     print(key, value)

In [ ]:
from src.cls_ner.cls_ner import get_query_engine

query_engine = get_query_engine()

In [ ]:
res = query_engine.query("港股票日行情中哪一个股票的最高价最高？")
print(res)

In [ ]:
res = query_engine.query("中国铁路通信信号有限公司发行前每股净资产为多少？")
print(res)

In [ ]:
res = query_engine.query("中国铁路通信信号有限公司发行前每股净资产为多少？发行后哪一天它的股价最高？")
print(res)

# Prompt

In [ ]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType


In [ ]:
for key, value in dict(DEFAULT_TEXT_TO_SQL_PROMPT).items():
    print(key, value)

In [ ]:
from src.prompt.text_to_sql_template import get_prompt
t2sql_template = get_prompt()

In [ ]:
text_to_sql_template = PromptTemplate(
    t2sql_template,
    prompt_type=PromptType.TEXT_TO_SQL,
    )

In [ ]:
for key, value in dict(text_to_sql_template).items():
    print(key, value)

# APP test

In [ ]:
import requests

base_url = "http://localhost:8888/api"

In [ ]:
# test rag
rag_response = requests.post(base_url + "/rag", json={"query": "中国铁路通信信号有限公司发行前每股净资产为多少？"})
print(rag_response.json())

In [ ]:
# test sql
sql_response = requests.post(base_url + "/sql", json={"query": "港股票日行情中哪一个股票的最高价最高？"})
print(sql_response.json())

In [ ]:
# test ner
ner_response = requests.post(base_url + "/ner", json={"query": "中国铁路通信信号有限公司发行前每股净资产为多少？"}).json().get("response")
print(ner_response["text"])